In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import sys

In [ ]:
import os

In [ ]:
import re

In [ ]:
basedir = "/home/marco/git/webvalley/datapreproc"
#basedir = "C:/Users/julix/webvalley_git/full_project/data/PLIC-Milano"
#basedir = "/home/mattia/Scrivania/plic_clinical_data/plic_clinical_data/PLIC-Milano"

# Import the smallest dataset

The plic-milano-foglio-piccolo.xlsx is PLIC_1445_V1_V2_V3_V4_linear variables _ updated 15_01_19.xlsx
and the plic-milano-foglio-grande.xlsx is PLIC_dataset parametri clinici_V1_V2_V3_V4_1445.xlsx

In [ ]:
smalldf = pd.read_excel(os.path.join(basedir, "plic-milano-foglio-piccolo.xlsx"), index_col=0).fillna(-1)

In [ ]:
smalldf.head()

In [ ]:
smalldf.info()

# What is male and what is female?
### Should at least be written in the docs of the whole software otherwise it will be impossible to import new data

In [ ]:
smalldf.sesso.unique()

# Import the biggest dataset

In [ ]:
bigdf = pd.read_excel(os.path.join(basedir, "plic-milano-foglio-grande.xlsx"), index_col=0).fillna(-1)

In [ ]:
bigdf.info()

In [ ]:
bigdf.head()

# At least one column is not in the biggest dataset

In [ ]:
smalldfcols = set(smalldf.columns.values)

In [ ]:
bigdfcols = set(bigdf.columns.values)

In [ ]:
f1_cols_in_f2 = smalldfcols.intersection(bigdfcols)

In [ ]:
f1_cols_non_in_f2 = smalldfcols.difference(bigdfcols)

In [ ]:
f1_cols_in_f2

In [ ]:
f1_cols_non_in_f2

In [ ]:
len(f1_cols_in_f2)

In [ ]:
len(f1_cols_non_in_f2)

Checking if column names differ because of case sensitivity

In [ ]:
bigdfcols_lower = set([k.lower() for k in bigdfcols])

In [ ]:
smalldfcols_lower = set([k.lower() for k in smalldfcols])

In [ ]:
len(smalldfcols_lower.intersection(bigdfcols_lower))

which is not the case

# Check how many patients in the smaller dataset are also in the biggest one

In [ ]:
paz_in_smalldf = set(smalldf.index.values)

In [ ]:
paz_in_bigdf = set(bigdf.index.values)

In [ ]:
paz_in_smalldf.difference(paz_in_bigdf)

In [ ]:
paz_in_bigdf.difference(paz_in_smalldf)

*All the patients in the small dataset are also in the biggest one*

In [ ]:
smalldf.head()

In [ ]:
bigdf.head()

# Fix known issues with the dataset

The two datasets uses different ways to store the data (M/F -> 0/1; Yes/No -> 1/0)

In [ ]:
smalldf["sesso"] = smalldf["sesso"].map({1: "F", 0: "M"})

*We dropped all sex errors so the method is appropriate; now replace even Si and No*

In [ ]:
# Check all the columns that contains at least one Si or No

bool_cols = list()

for col in f1_cols_in_f2:
    k = getattr(bigdf, col).values
    if np.isin("Sì", k) or np.isin("No", k):
        bool_cols.append(col)

In [ ]:
bool_cols

In [ ]:
# Replace em all

for col in bool_cols:
    smalldf[col] = smalldf[col].map({0: "No", 1: "Sì"})

In [ ]:
smalldf.head()

In [ ]:
bigdf.head()


## Now check if dataset 1 and dataset 2 shared patients contains same data

In [ ]:
# Pick every patient in the big dataset and check if the values are the same
# that are stored in the smaller one

for paz_to_be_checked in paz_in_smalldf:
    paz_from_big = bigdf.loc[paz_to_be_checked]
    paz_from_small = smalldf.loc[paz_to_be_checked]
    if not np.array_equal(paz_from_big[f1_cols_in_f2].values, paz_from_small[f1_cols_in_f2].values):
        print("Paz %s has differences" % paz_to_be_checked)

print("check complete")

# Adding non-duplicate data from smaller dataframe to bigger dataframe

## Preparing smaller dataframe to join with bigger dataframe

In [ ]:
smalldf.info()

In [ ]:
smalldf.drop(f1_cols_in_f2, axis=1, inplace=True)

In [ ]:
smalldf.info()

In [ ]:
bigdf.info()

In [ ]:
joined_df = bigdf.join(smalldf, how='outer')

In [ ]:
joined_df.info()
joined_df['menarca_1'].unique()

# Get rid of empty columns

In [ ]:
cols_to_drop = list()
for i in joined_df.columns.values:
    o = joined_df[i].unique()
    if len(o) == 1 and o[0] == -1:
        cols_to_drop.append(i)

cols_to_drop

In [ ]:
joined_df.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
joined_df.info()

# Remove columns ```_<int:pk>``` and divide patients visits on different rows

In [ ]:
[x for x in joined_df.columns]

In [ ]:
suffixes = ["_%s" % a for a in range(0,10)] + \
           ["_%s_a" % a for a in range(0,10)] + \
           ["_%s_recod" % a for a in range(0,10)]

In [ ]:
suffixes

In [ ]:
mv_cols = [x for x in joined_df.columns if x.endswith(tuple(suffixes))]

In [ ]:
mv_cols

In [ ]:
len(mv_cols)

In [ ]:
perdurant_cols = [x for x in joined_df.columns if x not in mv_cols]

In [ ]:
new_cols = set()

for col in mv_cols:
    for s in suffixes:
        col = col.replace(s, "")
    col.replace("__", "_")
    if col.endswith("_"):
        col = col[:-1]
    new_cols.add(col)

for col in perdurant_cols:
    new_cols.add(col)

new_cols.add("cod_pz")
new_cols

In [ ]:
len(new_cols)

# Create a new dataframe with the defined columns

In [ ]:
fields_per_visit = set()

for col in mv_cols:
    for s in suffixes:
        col = col.replace(s, re.sub("\d", "%s", s))
    fields_per_visit.add(col)

fields_per_visit

In [ ]:
# Test
for col in [a % b for a in fields_per_visit for b in range(1, 5)]:
    if col not in joined_df.columns.values:
        print("Cannot find", col)


In [ ]:
# Check we do have the same number of cols as before
len([k for k in [a % b for a in fields_per_visit for b in range(1, 5)] if k in joined_df.columns])

## Fill the new dataframe with the data from the other one

In [ ]:
# Build a dictionary to convert multicols names into singlecol
single = dict()

for col in mv_cols:
    _col = col
    for s in suffixes:
        col = col.replace(s, "")
    col.replace("__", "_")
    if col.endswith("_"):
        col = col[:-1]
    single[_col] = col

In [ ]:
all([k in new_cols for k in single.values()])

In [ ]:
new_data = []

for paz in joined_df.index.values:
        obj = joined_df.loc[paz]
        for visit in range(1, 5):
            this = {"cod_pz": paz}
            for old_col in fields_per_visit:
                try:
                    this[single[old_col % visit]] = obj[old_col % visit]
                except KeyError:
                    pass
            for pd_col in perdurant_cols:
                this[pd_col] = obj[pd_col]
            new_data.append(this)

In [ ]:
len(new_data)

In [ ]:
pippo = pd.DataFrame(new_data, columns=new_cols).fillna(-1)

In [ ]:
pippo.info()

In [ ]:
joined_df.info()

In [ ]:
pippo.head()

In [ ]:
k = set()
mask_fields = set()
for col in pippo.columns.values:
    if len(pippo[col].unique())<10:
        print(col, pippo[col].unique())
        mask_fields.add(col)

In [ ]:
rp = {
    "NR": -1,
    "campo vuoto": -1,
    "mancante": -1,
    "No": 0,
    "Sì": 1,
    "Si": 1,
    "no": 0,
    "sì": 1,
    "si": 1,
    "presenti": 1,
    "assenti": 0,
    "assente": 0,
    "completo": 1,
    "incompleto": 2,
    "sì - saltuario": 1,
    "sì - regolare": 2,
    "non palpabile": 0,
    "palpabile": 1,
    "normale": 0,
    "patologico": 1,
    "nessuna": 0,
    "leggera": 1,
    "media": 2,
    "pesante": 3,
    "ex": 2,
    "F": 1,
    "M": 0,
}

In [ ]:
for col in mask_fields:
    if pippo[col].dtype == np.object_:
        pippo[col] = pippo[col].replace(rp)

In [ ]:
pippo.head()

# Translate column names

In [ ]:
print("\n".join([x.replace("_", " ") for x in pippo]))

### Paste translated text


```
peripheral arteriopathy given limbs
DM PA1
hematocrit
Cute Xanthomas
pregnancies n
NS neoplasm
leukocytes
peripheral arteriopathy TSA
fabric g
IMT DC medium
ABI mean calc
peripheral arteriopathy TSA given
thyroid pathologies date
angina
Remote TIA to
aortic arteriopathy
extremities
blow type
angina profile
TG
cholecystectomy date
OT ATC1
HT PA1
HT indication1
PTCA date
AST
neoplasm2 operated
visits carried out
major arrhythmias date
COPD
vol cell medium
given atrial fibrillation
steatosis NS
epicardial fat
Remote IMA at
DM start date1
right corneal arch
major arrhythmias
DM ATC4
Dys start date2
diagnostic investigations3 outcome
arrhythmia date
OT ATC12
PA left ankle
OT ATC11
ischaemia silent profile
statins a
stroke date
neoplasia2 therapy
diagnostic investigations2 outcome
left plate NS
hypertension self
cholecystectomy
emergence
aorta calcifications
menopause
pad
PA ankle right
given neoplasm
HT PA4
LDL calc
monocytes
OT ATC2
HT ATC4
diagnostic investigations1 outcome
FE
b Beer
IMT CC max right
OT ATC3
cod pcs
glucose
remote silent ischemia a
ABI dx calc
other pathologies
Remote stroke at
self obesity
Dys ATC2
HT PA5
DM start date3
diagnosis of new revaluations
cute xanthelasms
diabetes1 self
IMT DC medium right
HT ATC5
abdomen type
diagnostic investigations1 date
PTCA profile
pas
fibers a
completed pregnancies
urine date
erythrocytes
PLACA
venous thrombosis given
cyanosis skin
ATT start date3
smoking
body scan data
HT ATC3
OT ATC8
height
waist hips calc
date of withdrawal
MVS
b wine
neoplasm1 NS
ginoid perc
xanthomatosis
b spirits
DM ATC1
RDW CV
NS blows
DM ATC3
neoplasm1 operated on
diabetes2 self
neoplasm2 given
remote angina a
abortion n
remote aortic arteriopathy a
diagnostic investigations1 type examination
BMI calc
HIGH
U TSA date
pancreas
Remote CHD to
skin hematomas
tofi skin
TIA profile
menarche
Android perc
thyroid function
neoplasm2 NS
OT ATC7
DM start date4
ages
flanks
burden
type neoplasm
ABI sx calc
abdomen
apoA1
HT PA3
diagnostic investigations2 type examination
EA Low
DM start date2
ATT PA2
ATT PA1
standardised visits to
Dys ATC1
cute oedemas
BMD g
self dyslipidaemia
clearance CG calc
Remote PTCA to
obesity self data
diagnostic investigations3 type examination
lean g
imbalance
IMT CC max left
thyroid pathologies text
learning
pancreatopathy type
left corneal arch
diagnostic investigations2 date
HT ATC1
OT ATC6
ECOSTEATOSIS ECOCARDIO
uricemia
ATT PA3
lifetime
EA
ATT indication1
alcohols
remote peripheral arteriopathy a
diabetes1 self data
neutrophils
lymphocytes
IMT CC medium left
SECONDARY PREVENTION
basophils percent
ATT start date2
NS Drugs
physical activity hours
IVS
heart rate
right plate NS
date endothelium
HT start date4
aortic arteriopathy date
hepatopathies type
HT start date3
ischemia silente date
remote decompensation at
physical activity intensity
diagnostic investigations3 date
neutrophils percent
endothelial dysfunction
peripheral arteriopathy limbs
bypass
HDL
hemoglobin cell medium
diabetes2 self data
known endothelium
neoplasm1 therapy
ATT start date1
thyroid appearance
gender
IMA profile
Profile bypass
Dys PA2
eosinophils
BMC g
self type dyslipidemia
enlarged liver
hepatopathies date
DM ATC2
date of visit
USED OT ATC10
plates
arrhythmia
HT PA2
thyroid nodules
basophils
bypass date
IVS date
OT ATC5
TIA
SIV
ATT ATC2
nephropathy date
pancreatopathy NS
haemoglobin
silent ischaemia
cerebral hemorrhage
fabric perc
IMT DC max
venous thrombosis
monocytes percent
respiratory system
angina date
peripheral arteriopathy
fat g
HT ATC2
induced
remote bypass to
pancreatopathy given
antiaggregants to
creatinine
RDW SD
eosinophils percent
OT ATC4
FMD
physical activity
menopause age
IMA date
DM PA4
atrial fibrillation
date TIA
apoB
gGT
skin infections
HT start date1
CHD
self data dyslipidemia
OT ATC9
COPD date
PCR
steatosis
nephropathies NS
AGRATIO
neoplasm2 type
ATT ATC3
IMA
antihypertensive to
steatosis degree
Stroke Profile
smoking recod
decompensation date
ATT ATC1
HT start date2
DM PA3
DM PA2
CPK
nephropathies profile
neoplasm1 date
cerebral hemorrhage given
hypertension self data
strokes
nephropathies type
well-known
pale skin
Lunar NS
peripheral arterial wrists
CHD profile
antidiabetics to
PTCA
neoplasm1 type
Dys PA1
conc hb cell medium
Dys start date1
hepatopathies NS
lymphocytes percent
TORQUE AND ANGLE WRENCH
peripheral arteriopathy given
```

In [ ]:
_ = """peripheral arteriopathy given limbs
DM PA1
hematocrit
Cute Xanthomas
pregnancies n
NS neoplasm
leukocytes
peripheral arteriopathy TSA
fabric g
IMT DC medium
ABI mean calc
peripheral arteriopathy TSA given
thyroid pathologies date
angina
Remote TIA to
aortic arteriopathy
extremities
blow type
angina profile
TG
cholecystectomy date
OT ATC1
HT PA1
HT indication1
PTCA date
AST
neoplasm2 operated
visits carried out
major arrhythmias date
COPD
vol cell medium
given atrial fibrillation
steatosis NS
epicardial fat
Remote IMA at
DM start date1
right corneal arch
major arrhythmias
DM ATC4
Dys start date2
diagnostic investigations3 outcome
arrhythmia date
OT ATC12
PA left ankle
OT ATC11
ischaemia silent profile
statins a
stroke date
neoplasia2 therapy
diagnostic investigations2 outcome
left plate NS
hypertension self
cholecystectomy
emergence
aorta calcifications
menopause
pad
PA ankle right
given neoplasm
HT PA4
LDL calc
monocytes
OT ATC2
HT ATC4
diagnostic investigations1 outcome
FE
b Beer
IMT CC max right
OT ATC3
cod pcs
glucose
remote silent ischemia a
ABI dx calc
other pathologies
Remote stroke at
self obesity
Dys ATC2
HT PA5
DM start date3
diagnosis of new revaluations
cute xanthelasms
diabetes1 self
IMT DC medium right
HT ATC5
abdomen type
diagnostic investigations1 date
PTCA profile
pas
fibers a
completed pregnancies
urine date
erythrocytes
PLACA
venous thrombosis given
cyanosis skin
ATT start date3
smoking
body scan data
HT ATC3
OT ATC8
height
waist hips calc
date of withdrawal
MVS
b wine
neoplasm1 NS
ginoid perc
xanthomatosis
b spirits
DM ATC1
RDW CV
NS blows
DM ATC3
neoplasm1 operated on
diabetes2 self
neoplasm2 given
remote angina a
abortion n
remote aortic arteriopathy a
diagnostic investigations1 type examination
BMI calc
HIGH
U TSA date
pancreas
Remote CHD to
skin hematomas
tofi skin
TIA profile
menarche
Android perc
thyroid function
neoplasm2 NS
OT ATC7
DM start date4
ages
flanks
burden
type neoplasm
ABI sx calc
abdomen
apoA1
HT PA3
diagnostic investigations2 type examination
EA Low
DM start date2
ATT PA2
ATT PA1
standardised visits to
Dys ATC1
cute oedemas
BMD g
self dyslipidaemia
clearance CG calc
Remote PTCA to
obesity self data
diagnostic investigations3 type examination
lean g
imbalance
IMT CC max left
thyroid pathologies text
learning
pancreatopathy type
left corneal arch
diagnostic investigations2 date
HT ATC1
OT ATC6
ECOSTEATOSIS ECOCARDIO
uricemia
ATT PA3
lifetime
EA
ATT indication1
alcohols
remote peripheral arteriopathy a
diabetes1 self data
neutrophils
lymphocytes
IMT CC medium left
SECONDARY PREVENTION
basophils percent
ATT start date2
NS Drugs
physical activity hours
IVS
heart rate
right plate NS
date endothelium
HT start date4
aortic arteriopathy date
hepatopathies type
HT start date3
ischemia silente date
remote decompensation at
physical activity intensity
diagnostic investigations3 date
neutrophils percent
endothelial dysfunction
peripheral arteriopathy limbs
bypass
HDL
hemoglobin cell medium
diabetes2 self data
known endothelium
neoplasm1 therapy
ATT start date1
thyroid appearance
gender
IMA profile
Profile bypass
Dys PA2
eosinophils
BMC g
self type dyslipidemia
enlarged liver
hepatopathies date
DM ATC2
date of visit
USED OT ATC10
plates
arrhythmia
HT PA2
thyroid nodules
basophils
bypass date
IVS date
OT ATC5
TIA
SIV
ATT ATC2
nephropathy date
pancreatopathy NS
haemoglobin
silent ischaemia
cerebral hemorrhage
fabric perc
IMT DC max
venous thrombosis
monocytes percent
respiratory system
angina date
peripheral arteriopathy
fat g
HT ATC2
induced
remote bypass to
pancreatopathy given
antiaggregants to
creatinine
RDW SD
eosinophils percent
OT ATC4
FMD
physical activity
menopause age
IMA date
DM PA4
atrial fibrillation
date TIA
apoB
gGT
skin infections
HT start date1
CHD
self data dyslipidemia
OT ATC9
COPD date
PCR
steatosis
nephropathies NS
AGRATIO
neoplasm2 type
ATT ATC3
IMA
antihypertensive to
steatosis degree
Stroke Profile
smoking recod
decompensation date
ATT ATC1
HT start date2
DM PA3
DM PA2
CPK
nephropathies profile
neoplasm1 date
cerebral hemorrhage given
hypertension self data
strokes
nephropathies type
well-known
pale skin
Lunar NS
peripheral arterial wrists
CHD profile
antidiabetics to
PTCA
neoplasm1 type
Dys PA1
conc hb cell medium
Dys start date1
hepatopathies NS
lymphocytes percent
TORQUE AND ANGLE WRENCH
peripheral arteriopathy given""".split("\n")

In [ ]:
assert len([a for a in pippo]) == len(_)

In [ ]:
pippo.head()

In [ ]:
pippo.columns = _

In [ ]:
pippo.head()

In [ ]:
pippo.to_csv("/tmp/out_Milano.csv")

# Translate variable names (feature temporarilyfrozen)

In [ ]:
print("The end.")